## Summarization

we try to summarize on the [xsum](https://huggingface.co/datasets/knkarthick/xsum) dataset .

In [1]:
!pip install sentencepiece
!pip install 'transformers[torch]'
!pip install zstandard
!pip install sacrebleu
!pip install rouge_score
!pip install -U accelerate
!pip install -U transformers
!pip install -U datasets
!pip install jupyterlab_widgets
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
from datasets import load_dataset, DatasetDict
#!pip install tensorflow_probability==0.12.2
from transformers import pipeline
import nltk
import os
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from datasets import load_metric
import pandas as pd
import numpy as np
import torch
#from tqdm import tqdm
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/gabriel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
# Load the XSum dataset
ds = load_dataset("knkarthick/xsum", split=None)
print 
# Access each split directly
train_set = ds['train']
valid_set = ds['validation']
test_set = ds['test']

In [53]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 11334
    })
})

In [54]:
ds["train"][1]

{'id': 40143035,
 'dialogue': 'A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.\nAs they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.\nOne of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.\nThe driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.\nBoth groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.\nPolice have appealed for information about the attack.\nInsp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.\n"While the exact cause is still under investigation, it is thought that the fire was started deliberately."',
 'summary': 'Two tourist buses have been destroyed by fire in a suspected ars

In [55]:
sample_text = ds["train"][1]["dialogue"][:2000]
sample_text


'A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.\nAs they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.\nOne of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.\nThe driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.\nBoth groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.\nPolice have appealed for information about the attack.\nInsp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.\n"While the exact cause is still under investigation, it is thought that the fire was started deliberately."'

#Generating a baseline summary taking the first 3 sentences

In [56]:
def get_baseline_summary(text, num_sentences=3):
    sentences = sent_tokenize(text)
    return ' '.join(sentences[:num_sentences])

In [57]:
summaries = {}

In [58]:
summaries["baseline"] = get_baseline_summary(sample_text)

### GPT2
Can implement a summarization by using the text-generation pipeline but appending a "TL:DR" at the end of the prompt

In [11]:
pipe = pipeline("text-generation", model="gpt2") #If gpt2-xl is too large use a small version
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))
summaries["gpt2"]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\nAt one time, three buses were parked side-by-side in the Carpark side of Belfast, in the wake of the horrific event.\nThe buses were to be replaced by buses from the other two hotels in the area.\nThey were then to leave Belfast, the Northern Ireland Transport Authority said in a statement.\nThe vans were supposed to have arrived at Derry, but the fire that engulfed them was brought to the attention of the Northern Ireland fire service.'

### T5

In [12]:
pipe = pipeline("summarization", model="t5-small") #If t5-large is too large use t5-small version
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["t5"]

Your max_length is set to 200, but your input_length is only 194. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=97)


'a fire alarm went off at the Holiday Inn in Hope Street on Saturday .\nguests were asked to leave the hotel .\nthe two buses were parked side-by-side in the car park .'

## BART

In [13]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["bart"]

'Fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST.\nOne of the tour groups is from Germany, the other from China and Taiwan.\nThe driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.'

In [14]:
print("GROUND TRUTH")
print(ds["train"][1]["summary"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Two tourist buses have been destroyed by fire in a suspected arson attack in Belfast city centre.

BASELINE
A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel. As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames. One of the tour groups is from Germany, the other from China and Taiwan.

GPT2

At one time, three buses were parked side-by-side in the Carpark side of Belfast, in the wake of the horrific event.
The buses were to be replaced by buses from the other two hotels in the area.
They were then to leave Belfast, the Northern Ireland Transport Authority said in a statement.
The vans were supposed to have arrived at Derry, but the fire that engulfed them was brought to the attention of the Northern Ireland fire service.

T5
a fire alarm went off at the Holiday Inn in Hope Street on Saturday .
guests were asked to leave the hotel .
the two buses

## Evaluation

How can we evaluate the performance of a summarization model?

**BLEU (Bilingual Evaluation Understudy) Score**  is a precision based metric that measures how many words or n-grams present in the machine generated summaries also appear in the human reference summaries divided by the length of the generation. It penalises the appearance of too short summaries with a brevity penalty. [Sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu).

**ROUGE (Recall Oriented Understudy for Gisting Evaluation) score** is the recall based metric that measures how many words/ngrams present in the human reference summaries were found in the machine generated summaries. Some sub-forms:

1. ROUGE-N: Overlap of n-grams
2. ROUGE-1: Overlap of unigrams.
3. ROUGE-2: Oerlap of bigrams.
4. ROUGE-L: Overlap of the Longest Common Subsequence

To Do:
1. How do you interpret a Bleu/ Rouge Score of 0?

In [15]:
bleu_metric = load_metric("sacrebleu")

In [16]:
rouge_metric = load_metric("rouge")

In [17]:
reference = ds["train"][1]["summary"]
reference

'Two tourist buses have been destroyed by fire in a suspected arson attack in Belfast city centre.'

In [18]:
bleu_scores = []
for model_name in summaries:
  bleu_metric.add(prediction=summaries[model_name], reference=[reference])
  results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
  bleu_scores.append(results["score"])
  #results["precisions"] = [np.round(p, 2) for p in results["precisions"]]

In [19]:

records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    rouge_score = rouge_metric.compute()
    result_dict = dict((rn, rouge_score[rn].mid.fmeasure) for rn in rouge_names)
    bleu_metric.add(prediction=summaries[model_name], reference=[reference])
    bleu_results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
    result_dict["bleu"] = bleu_results["score"]
    records.append(result_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum,bleu
baseline,0.179487,0.000000,0.102564,0.102564,0.0
gpt2,0.185567,0.021053,0.103093,0.144330,0.0
t5,0.280000,0.000000,0.160000,0.200000,0.0
bart,0.142857,0.029412,0.085714,0.142857,0.0


BASELINE: some overlap of individual words, but bad performance on word pairs and no overlap (BLEU)

GPT2: captured some bigrams (rouge2) but the bleu score indicates still no overlaps

T5: best capturing in unigrams, but still no real performance in higher overlaps

## Finetuning a model for Summarization (BERT)

#Discussion
How would you fine tune a model for summarization?

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Truncation of input to 1024 characters and 161 target summary size due to the training data has 95% of data below 161 characters here.



In [60]:
def convert_data_to_features(batch, tokenizer, max_input_length=1024, max_target_length=161):
    input_encodings = tokenizer(batch['dialogue'], padding="max_length", truncation=True, max_length=max_input_length)
    target_encodings = tokenizer(batch['summary'], padding="max_length", truncation=True, max_length=max_target_length)

    return {
    "input_ids": input_encodings.input_ids,
    "attention_mask": input_encodings.attention_mask,
    "labels": target_encodings.input_ids
    }
    #with tokenizer.as_target_tokenizer(): #as_target_tokenizer helps to differentiate between
    #    target_encodings = tokenizer(batch["summary"], max_length=max_target_length, truncation=True)

    #return {"input_ids": input_encodings["input_ids"],
    #        "attention_mask": input_encodings["attention_mask"],
    #        "labels": target_encodings["input_ids"]}

In [61]:
# Filter out rows where either 'dialogue' or 'summary' is None
filtered_ds = ds.filter(lambda x: x['dialogue'] is not None and x['summary'] is not None)

# Applying the convert_data_to_features function to the filtered dataset
dataset_xsum = filtered_ds.map(lambda batch: convert_data_to_features(batch, tokenizer), batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_xsum.set_format(type="torch", columns=columns)


Map:   0%|          | 0/204017 [00:00<?, ? examples/s]

Map:   0%|          | 0/11327 [00:00<?, ? examples/s]

Map:   0%|          | 0/11333 [00:00<?, ? examples/s]

In [24]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='output_folder', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=False,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

In [26]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_xsum["train"],
                  eval_dataset=dataset_xsum["validation"])

In [27]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.439900,0.370914
1000,0.415000,0.360295
1500,0.420800,0.354789
2000,0.383300,0.349558
2500,0.388200,0.346127
3000,0.381000,0.342745
3500,0.406100,0.340163
4000,0.381100,0.338319
4500,0.356600,0.336035
5000,0.378900,0.334091


TrainOutput(global_step=12751, training_loss=0.47091856156645656, metrics={'train_runtime': 63066.8981, 'train_samples_per_second': 3.235, 'train_steps_per_second': 0.202, 'total_flos': 1.2439599326429184e+17, 'train_loss': 0.47091856156645656, 'epoch': 1.0})

In [64]:
test_dataset=dataset_xsum["test"]

def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries(test_dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="dialogue",
                               column_summary="summary"):
    article_batches = list(chunks(test_dataset,[column_text], batch_size))
    target_batches = list(chunks(test_dataset,[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=161)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    print("Score:", score)
    return score

In [47]:

# Function to find max length of a field in a dataset split, handling None values
#def find_max_length(dataset, field):
#    return max(len(entry[field]) if entry[field] is not None else 0 for entry in dataset)


# Calculate max length for 'dialogue' and 'summary' in each split
#max_length_train_dialogue = find_max_length(ds['train'], 'dialogue')
#max_length_train_summary = find_max_length(ds['train'], 'summary')

#max_length_val_dialogue = find_max_length(ds['validation'], 'dialogue')
#max_length_val_summary = find_max_length(ds['validation'], 'summary')

#max_length_test_dialogue = find_max_length(ds['test'], 'dialogue')
#max_length_test_summary = find_max_length(ds['test'], 'summary')

# Print the results
#print("Max length in 'train' - Dialogue:", max_length_train_dialogue, ", Summary:", max_length_train_summary)
#print("Max length in 'validation' - Dialogue:", max_length_val_dialogue, ", Summary:", max_length_val_summary)
#print("Max length in 'test' - Dialogue:", max_length_test_dialogue, ", Summary:", max_length_test_summary)


In [65]:
def evaluate_summaries(dataset, metric, model, tokenizer, batch_size, device, column_text="dialogue", column_summary="summary"):
    # Ensure dataset is filtered and preprocessed similarly to training/validation sets
    dataset = dataset.filter(lambda x: x[column_text] is not None and x[column_summary] is not None)
    dataset = dataset.map(lambda batch: convert_data_to_features(batch, tokenizer), batched=True)
    dataset.set_format(type="torch", columns=["input_ids", "labels", "attention_mask"])

    # Rest of your existing evaluation logic...


In [66]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [69]:
rouge_score = evaluate_summaries(test_dataset, rouge_metric, trainer.model, tokenizer,batch_size=2, device=device)
# Debugging: print the rouge_score to see what it contains
print("Rouge Score:", rouge_score)

Filter:   0%|          | 0/11333 [00:00<?, ? examples/s]

Map:   0%|          | 0/11333 [00:00<?, ? examples/s]

Rouge Score: None


In [70]:
rouge_dict = dict((rn, rouge_score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"cnn"])

TypeError: 'NoneType' object is not subscriptable

## BLEU score
How would you calculate the BLEU score?

In [82]:
# functional testing on a subset
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm

def calculate_bleu_score(dataset, model, tokenizer, device, subset_size=100, batch_size=16, column_text="dialogue", column_summary="summary"):
    # Select a subset of the dataset for testing
    dataset_subset = dataset.select(range(subset_size))
    references = []
    candidates = []

    for batch in tqdm(chunks(dataset_subset, batch_size)):
        #debug
        print(f"Processing batch of size: {len(batch[column_text])}")
        
        # Ensure batch[column_text] is a list of strings
        input_texts = [text for text in batch[column_text] if isinstance(text, str)]
        reference_texts = [ref for ref in batch[column_summary] if isinstance(ref, str)]

        # Adjust batch size based on filtered data
        actual_batch_size = len(input_texts)
        if actual_batch_size == 0:
            continue

        # Tokenizing the input text
        inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(device)

        # Generating summaries
        summaries = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=161, num_beams=8)
        
        # Decoding the generated summaries
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]

        # Ensure that we have the same number of references and candidates
        decoded_summaries = decoded_summaries[:actual_batch_size]
        reference_texts = reference_texts[:actual_batch_size]

        # Debugging: Print a sample summary and its reference
        if len(decoded_summaries) > 0:
            print("Sample generated summary:", decoded_summaries[0])
            print("Corresponding reference summary:", batch[column_summary][0])

        
        # Adding to candidates and references
        candidates.extend(decoded_summaries)
        references.extend([[ref.split()] for ref in reference_texts])

    # Ensure equal number of candidates and references
    assert len(candidates) == len(references), "Number of candidates and references must be equal"

    # Calculating BLEU score
    bleu_score = corpus_bleu(references, [c.split() for c in candidates])
    return bleu_score

# Testing with a subset
bleu_score_subset = calculate_bleu_score(ds["test"], trainer.model, tokenizer, device, subset_size=100, batch_size=2)
print("BLEU Score on Subset:", bleu_score_subset)


0it [00:00, ?it/s]

Processing batch of size: 2


1it [00:00,  1.55it/s]

Sample generated summary: More than 1,000 new one-bedroom flats in Wales have been built in the past year, according to a charity.
Corresponding reference summary: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
Processing batch of size: 2


2it [00:01,  1.94it/s]

...

Sample generated summary: The UK has voted to leave the European Union.
Corresponding reference summary: Economic policymakers must now decide whether and how they should respond to the UK's vote to leave the EU.
Processing batch of size: 2


49it [00:28,  1.73it/s]

Sample generated summary: A health board has apologised to the families of patients who died after being treated like animals at a psychiatric ward.
Corresponding reference summary: The quality of care on a scandal-hit ward for dementia patients may have contributed to at least seven deaths, BBC Wales can reveal.
Processing batch of size: 2


50it [00:29,  1.71it/s]

Sample generated summary: US women's team-mate Hope Solo has been suspended for three matches after testing positive for a banned substance.
Corresponding reference summary: The United States women's team goalkeeper Hope Solo has been suspended for 30 days by US Soccer following an incident during a training camp.


In [91]:
# evaluating real score
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm

def calculate_bleu_score(dataset, model, tokenizer, device, subset_size=100, batch_size=16, column_text="dialogue", column_summary="summary"):
    # Select a subset of the dataset for testing
    #dataset_subset = dataset.select(range(subset_size))
    references = []
    candidates = []

    for batch in tqdm(chunks(dataset, batch_size)):
        #debug
        #print(f"Processing batch of size: {len(batch[column_text])}")
        
        # Ensure batch[column_text] is a list of strings
        input_texts = [text for text in batch[column_text] if isinstance(text, str)]
        reference_texts = [ref for ref in batch[column_summary] if isinstance(ref, str)]

        # Adjust batch size based on filtered data
        actual_batch_size = len(input_texts)
        if actual_batch_size == 0:
            continue

        # Tokenizing the input text
        inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(device)

        # Generating summaries
        summaries = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=161, num_beams=8)
        
        # Decoding the generated summaries
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]

        # Ensure that we have the same number of references and candidates
        decoded_summaries = decoded_summaries[:actual_batch_size]
        reference_texts = reference_texts[:actual_batch_size]

        # Debugging: Print a sample summary and its reference
        #if len(decoded_summaries) > 0:
        #    print("Sample generated summary:", decoded_summaries[0])
        #    print("Corresponding reference summary:", batch[column_summary][0])

        
        # Adding to candidates and references
        candidates.extend(decoded_summaries)
        references.extend([[ref.split()] for ref in reference_texts])

    # Ensure equal number of candidates and references
    assert len(candidates) == len(references), "Number of candidates and references must be equal"

    # Calculating BLEU score
    bleu_score = corpus_bleu(references, [c.split() for c in candidates])
    return bleu_score

# Testing with a subset
bleu_score = calculate_bleu_score(ds["test"], trainer.model, tokenizer, device, subset_size=100, batch_size=2)
print("BLEU Score:", bleu_score)
#5500it

5667it [1:17:29,  1.22it/s]


BLEU Score: 0.09534446192887946


In [90]:
# free some memory
import gc
gc.collect()


7668

In [95]:
def generate_summary(input_text, model, tokenizer, device):
    # Tokenize the input text
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate summary
    summary_ids = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], 
                                 max_length=161, num_beams=5, length_penalty=2.0, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# TRAIN SAMPLE
input_dialogue_train = "In a Commons debate, the ex-Labour leader said Conservative plans to limit the voting powers of Scottish MPs on Commons laws would rip up hundreds of years of parliamentary procedure. The SNP said the Conservatives wanted to create a quasi-English Parliament. But ministers said it was vital England was treated fairly as further powers were devolved to other parts of the UK. At the end of the debate, Labour staged and won a vote in which the government abstained. And Conservative MP David Davis raised a point of order to urge the government to allow more time for the matter to be considered. The government believes bills applying exclusively to England should not become law without the explicit consent of MPs from English constituencies and it wants to change Commons rules known as standing orders to give them a decisive say during their passage. Ministers say this will address the longstanding anomaly by which Scottish MPs can vote on issues such as health and education affecting England but English MPs have no say on similar matters relating to Scotland, where such policies are devolved. Mark D'Arcy, BBC Parliamentary correspondent Panic stations? From the point of view of the government whips this afternoon's emergency Commons debate on English Votes for English Laws was really rather alarming. The debate, so skilfully secured by Lib Dem ex Scottish secretary Alistair Carmichael, produced a slightly pointless vote in which the Conservatives mostly abstained, but it brought all kinds of nasty tensions to the surface. Conservative MPs are supposed to be signed up for EVEL under the terms of their manifesto, but there were clearly quite a few with doubts, some about the policy, many more about the process, which was Mr Carmichael's line of attack. Read the article in full However, the plans came under sustained attack from Labour, SNP and Lib Dems in an urgent debate, secured by former Scottish Secretary Alistair Carmichael. MPs are due to debate and vote on the government's plans next week but Mr Carmichael, the MP for Orkney and Shetland, said the plans required more detailed consideration, arguably through primary legislation. If MPs representing constituencies in England had a veto on certain legislation, it would breach the longstanding principle that MPs, no matter who they represented, were all equal. To go as far as the government wants to go in the timescale they want to go brings with it an attendant level of risk that I would consider to be irresponsible, he said. They are not entitled to use the UK Parliament as a proxy for an English Parliament. Assurances that the new system would be reviewed by MPs after a year were inadequate, he added, saying this would not be capable of putting a dangerous genie back in the bottle after it had been let out - we all know that is the political reality. But Commons Leader Chris Grayling said it was simply incorrect to claim some MPs would be prevented from debating and voting on certain legislation and would continue to exercise the same rights as they do now. Pressed by Mr Miliband on what the meaning and definition of English-only legislation would be, Mr Grayling said it would be up to the Speaker to decide but he believed a simple test would be what things were devolved to Scotland. Requiring bills to have the support of a double majority of the whole of the Commons and those MPs representing England would also help mitigate any resentment felt by English voters about the slower pace of devolution to England. It is of vital importance that English citizens of the UK, as we move to an extra layer of devolution to Scotland and Wales and devolve additional tax powers to Northern Ireland, that they think it is fair, he said. It is what we pledged to do in our manifesto. We set it out in detail, step by step by step. We are implementing these changes and keeping our promises. I think the people who elected us would expect nothing else. Mr Miliband said the Conservatives had the power to address the issue after winning the election but urged them to think again. Is this true to the traditions of Conservatism? No because the last thing you do is rip up hundreds of years of constitutional practice in a standing order vote just before the House goes into recess, he said. Doing this procedure in the way it is being proposed is an act of constitutional vandalism. It really is. The SNP said Scottish voters would be affected by legislation on schools and NHS budgets in England through the Barnett Formula used to allocate public spending to different nations of the UK. This not just English votes for English laws, this is English votes for Scottish laws, said Pete Wishart, MP for Perth and North Perthshire. It is totally and utterly unacceptable. He added: Why don't they just tattoo our foreheads 'Scottish' and then they would be able to identify us. And Conservative MP Sir Edward Leigh said the move would not make any difference to the outcome of votes and urged ministers to omit laws which indirectly applied to Scotland, saying not to do so would be a gift to the SNP's independence campaign. Responding to a question by an SNP MP on Monday, Mr Grayling said the only English-only measure during the last Parliament had been the Education Bill, and there were 13 Bills which applied to England and Wales."  # Replace with your dialogue
summary_train = generate_summary(input_dialogue_train, trainer.model, tokenizer, device)
print("DIALOGUE INPUT :", input_dialogue_train)
print("SUMMARY TRAIN (:", summary_train)

# VALIDATION SAMPLE
input_dialogue_val = "Mr Bondevik, who said he was travelling on his diplomatic passport, was held for about an hour. He said his passport also indicates he is a former prime minister. Immigration officials told him it was unrelated to President Trump's temporary ban on Iranian nationals. Instead, he was told it related to a 2015 law which places extra restrictions on countries that are part of the US visa waiver programme, according to an interview with ABC7 news. But Mr Bondevik said he has never had an issue travelling to the US with the same document before Mr Trump's order. During his 2014 Iran trip, he spoke against extremism at an international conference on behalf of human rights organisation The Oslo Centre, of which he is president. Iran is one of the seven countries affected by the controversial executive order from the new president. I was surprised, and I was provoked, he said, suggesting that the mention of Iran had made him stick out. There is no reason to be afraid of a former head of government who has been on official visits several times to this country, including in the White House, he told Norwegian broadcaster TV2. The former prime minister was flying to the US to attend the national prayer breakfast event in Washington - which President Donald Trump also attended."  # Replace with your dialogue
summary_val = generate_summary(input_dialogue_val, trainer.model, tokenizer, device)
print("DIALOGUE INPUT :", input_dialogue_val)
print("SUMMARY VALIDATION (:", summary_val)

# TEST SAMPLE
input_dialogue_test = "Tawel Fan ward at Glan Clwyd Hospital, Denbighshire, was closed more than three years ago and a report found some patients were treated like animals. It has emerged that at least seven patients' families were told treatment may have contributed to their deaths. Betsi Cadwaladr health board said an investigation was under way. It acknowledged the quality of care provided could have been a contributory factor to the deaths of some patients. A review of mortality rates on the ward has never been published although it is understood it has been completed. Relatives of one patient told BBC Wales Today they were told medical care on the ward was inadequate. Correspondence seen by the programme included an apology from the health board to the family, who do not want to be identified. One letter said: Experts found that there were problems in the health care which may have contributed to the death. It added that the board is very much engaged in a thorough search for the truth about the Tawel Fan ward. But the family were unconvinced lessons had been learned and said questions remained unanswered and, as far as they were aware, nobody had lost their job, let alone be prosecuted. The scandal of Tawel Fan pushed the already troubled health board into close supervision by the Welsh Government. It remains in special measures which costs Â£5m a year. An initial report into what happened at Tawel Fan was published almost three years ago. Two more reports are due later this year. One of them, being compiled by the Health and Social Care Advisory Service (Hascas), is expected to include details of a mortality review of Tawel Fan patients. But Geoff Ryall-Harvey, who leads the patient watchdog Community Health Council in north Wales, said it should be released as soon as possible. It may stop this practice elsewhere, he added. A Betsi Cadwaladr health board spokesman said: We acknowledge that the quality of care provided could have been a contributory factor to the death of some patients. Whether this is the case will be established as part of the independent Hascas investigation, which is currently being carried out. In order to establish whether or not the quality of care contributed to any patients' death, every aspect of every patient's care has to be investigated. This is a complicated and time consuming process, but must be carried out in order to determine whether or not the care provided was a contributory factor to any patients' death. Every family involved in the investigation will receive an individual report detailing the care provided to their relative. These reports will also help inform the findings of the Tawel Fan investigation."  # Replace with your dialogue
summary_test = generate_summary(input_dialogue_test, trainer.model, tokenizer, device)
print("DIALOGUE INPUT :", input_dialogue_test)
print("SUMMARY TEST (:", summary_test)

DIALOGUE INPUT : In a Commons debate, the ex-Labour leader said Conservative plans to limit the voting powers of Scottish MPs on Commons laws would rip up hundreds of years of parliamentary procedure. The SNP said the Conservatives wanted to create a quasi-English Parliament. But ministers said it was vital England was treated fairly as further powers were devolved to other parts of the UK. At the end of the debate, Labour staged and won a vote in which the government abstained. And Conservative MP David Davis raised a point of order to urge the government to allow more time for the matter to be considered. The government believes bills applying exclusively to England should not become law without the explicit consent of MPs from English constituencies and it wants to change Commons rules known as standing orders to give them a decisive say during their passage. Ministers say this will address the longstanding anomaly by which Scottish MPs can vote on issues such as health and educatio

In [101]:
def calculate_rouge_scores_subset(dataset, model, tokenizer, device, subset_size=100, batch_size=16, column_text="dialogue", column_summary="summary"):
    # Select a subset of the dataset for testing
    dataset_subset = dataset.select(range(subset_size))

    references = []
    candidates = []

    for i in range(0, subset_size, batch_size):
        # Prepare batch
        batch_texts = dataset_subset[column_text][i:i+batch_size]
        batch_summs = dataset_subset[column_summary][i:i+batch_size]

        # Tokenize inputs and generate summaries
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
        summaries = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=161, num_beams=5)

        # Decode summaries
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]

        # Add to references and candidates
        candidates.extend(decoded_summaries)
        references.extend([[ref.split()] for ref in batch_summs])

    # Ensure equal number of candidates and references
    assert len(candidates) == len(references), "Number of candidates and references must be equal"

    # Calculating BLEU score
    rouge_scores = rouge.compute(predictions=candidates, references=references)
    return rouge_scores

# Testing with a subset
rouge_scores_subset = calculate_rouge_scores_subset(ds["test"], trainer.model, tokenizer, device, subset_size=100, batch_size=2)
print("ROUGE Scores on Subset:", rouge_scores_subset)


ROUGE Scores on Subset: {'rouge1': AggregateScore(low=Score(precision=0.4315830088241217, recall=0.3574231608428432, fmeasure=0.38385465959289244), mid=Score(precision=0.43502798335377973, recall=0.3601564009121777, fmeasure=0.3867002135077965), high=Score(precision=0.43807216319982695, recall=0.36271897967657923, fmeasure=0.3892429940535571)), 'rouge2': AggregateScore(low=Score(precision=0.18801655397960232, recall=0.154659037656873, fmeasure=0.1664991011804962), mid=Score(precision=0.19085409171130258, recall=0.15714728175533343, fmeasure=0.16907595361147953), high=Score(precision=0.19393223202573878, recall=0.15975815411422092, fmeasure=0.1717838627643787)), 'rougeL': AggregateScore(low=Score(precision=0.3514365853320121, recall=0.29149576731545773, fmeasure=0.31279052732592283), mid=Score(precision=0.3544319029068154, recall=0.29417114489588203, fmeasure=0.3154797325664329), high=Score(precision=0.3579650710201704, recall=0.29702822650632227, fmeasure=0.31852707532764757)), 'rougeL